# Intersection of OO and Functional Programming

- Python supports many features of pure functional programming languages such as Lisp
- Python also supports many features of OOP of pure OOP languages such as Java, C#
- in his chapter, we'll learn a grab bag of Python features that are not purely object-oriented:

    - built-in functions that take care of common tasks in one call
    - an alternative to method-overloading
    - functions as objects
    - file I/O and context managers
    
## Python built-in functions

- all A-Z built-in functions are listed here: https://docs.python.org/3/library/functions.html


### len( )
    - calls `__len__()` method of objects
    - len() is better decision choice than calling .len() or .__len__()
    - works the same way on all containers!
    - easy to understand, remember, use, etc.

In [25]:
help("builtins")

Help on built-in module builtins:

NAME
    builtins - Built-in functions, exceptions, and other objects.

DESCRIPTION
    Noteworthy: None is the `nil' object; Ellipsis represents `...' in slices.

CLASSES
    object
        BaseException
            Exception
                ArithmeticError
                    FloatingPointError
                    OverflowError
                    ZeroDivisionError
                AssertionError
                AttributeError
                BufferError
                EOFError
                ImportError
                    ModuleNotFoundError
                LookupError
                    IndexError
                    KeyError
                MemoryError
                NameError
                    UnboundLocalError
                OSError
                    BlockingIOError
                    ChildProcessError
                    ConnectionError
                        BrokenPipeError
                        ConnectionAbortedError
           

In [1]:
len([1, 2, 3, 3])

4

In [2]:
len(set([1,2 ,3, 3, 3, 4]))

4

### reversed ( )

- takes a sequence as an input and returns the copy of the sequence in reverse order
- normally used in for statements when we want to iterate items from back to front
- similar to `len()`, `reversed()` calls `__reversed__()` of objects
    - if `__reversed__()` is not found, it uses `__len__()` and `__getitem__()` to build the reversed sequence
- returns reversed iterator that yields one element at a time
- we'll learn more about iterators in Iterator design pattern later

In [4]:
generic = [1, 2, 3, 4, 5]

In [18]:
print(reversed(generic))

In [19]:
print(list(reversed(generic)))

[5, 4, 3, 2, 1]


In [11]:
class CustomClass:
    def __init__(self, args):
        self._list = args
        
    def __len__(self):
        return 5
    
    # if either __len__ or __getitime__ is missing; you can't call reversed
    def __getitem__(self, idx):
        return f'x{idx}'

In [12]:
custom = CustomClass([1, 2, 3, 4, 5])

In [13]:
print(list(reversed(custom)))

['x4', 'x3', 'x2', 'x1', 'x0']


In [14]:
class FunkyBackwards(list):
    
    # override __reversed__ of list class
    def __reversed__(self):
        return 'BACKWARDS!'

In [15]:
funky = FunkyBackwards([1, 2, 3, 4, 5])

In [17]:
print(reversed(funky))

BACKWARDS!


### enumerate( )

- sometimes we need to know both the index and value from a sequence while iterating over
- enumerate() will helps us do just that

In [20]:
numbers = [10, 9, 100, 15, 20]

In [23]:
for i, val in enumerate(numbers):
    print(f'{i}: {val}')

0: 10
1: 9
2: 100
3: 15
4: 20


### all( )

- all takes an iterable object and evaluates to true if all the elements are true
    - returns False if a single element evaluates to false

In [26]:
all([True, True, True])

True

In [27]:
all([True, True, False])

False

### any( )

- returns true if any of the elements in the provided iterator is True
- False if all the elements are False

In [28]:
any([True, True, False])

True

In [29]:
any([False, False, False])

False

### many more...

- abs( ), str( ), repr( ), pow( ), and divmod( ) map directly to the special methods `__abs__()`, `__str__()`, `__repr__()`, `__pow__()`, and `__divmod__()`
- bytes( ), format( ), hash( ), and bool( ) also map directly to the special methods `__bytes__()`, `__format__()`, `__hash__()`, and `__bool__()`

## Alternative to method overloading

- in functional programming, we call it function overloading
- sometime the same method needs to act on data of different types
- in static-typed language such as C++, we redefine functions with various parameters and types
     - or use templated prameters to accomodate for various types
- in dynamic-typed language we don't need to overload for types!
     - this can introduce errors, however!
- two types of overloading:
    1. overloading parameters to allow alternative types
        - using Union[...] type hints for mypy
    2. overlaoding parameters to allow variable number of arguments

In [30]:
def add(n1, n2):
    return n1+n2

In [31]:
add(10, 20)

30

In [32]:
add(10.5, 20.5)

31.0

In [33]:
add('hi', 'there')

'hithere'

In [34]:
# missing required parameters
add()

TypeError: add() missing 2 required positional arguments: 'n1' and 'n2'

In [11]:
# Any is truely generic type!
from typing import Any

def mandatory_params(x: Any, y: Any, z: Any) -> str:
    return f"{x=}, {y=}, {z=}"

In [13]:
mandatory_params(1, 2.5, 'Hello')

"x=1, y=2.5, z='Hello'"

## Default values for parameters

- sometimes we've to provide default values for parameters making them optional
- long list of positional parameters isn't a great idea
    - it can be confusing

In [14]:
from typing import Optional

In [15]:
# the last two parameters have default values and can be omitted
def latitude_dms(
    deg: float, minute: float, sec: float = 0.0, direction: Optional[str] = None
) -> str:
    if direction is None:
        direction = "N"
    return f"{deg:02.0f}° {minute+sec/60:05.3f}{direction}"

In [16]:
latitude_dms(36, 51, 2.9, "N")

'36° 51.048N'

In [17]:
latitude_dms(36, 51)

'36° 51.000N'

In [18]:
latitude_dms(38, 58, direction="N")

'38° 58.000N'

In [19]:
# we can reorder the parameters, provided the values are provided
latitude_dms(38, 19, direction="N", sec=7)

'38° 19.117N'

In [20]:
latitude_dms(minute=38, deg=19, direction="N", sec=7)

'19° 38.117N'

In [21]:
# place * before keyword only parameters
# a and b are keyword only parameters
def kw_only(
    x: Any, y: str = "defaultkw", *, a: bool, b: str = "only"
) -> str:
    return f"{x=}, {y=}, {a=}, {b=}"

In [22]:
kw_only('x')

TypeError: kw_only() missing 1 required keyword-only argument: 'a'

In [23]:
kw_only('x', 'y', 'a')

TypeError: kw_only() takes from 1 to 2 positional arguments but 3 were given

In [24]:
kw_only('x', a='a', b='b')

"x='x', y='defaultkw', a='a', b='b'"

In [25]:
# can also mark parameters as being supplied only by position
# use / to separate position only from more flexible ones
def pos_only(x: Any, y: str, /, z: Optional[Any] = None) -> str:
    return f"{x=}, {y=}, {z=}"

In [26]:
# can't position-only arguments as keyword arguments
pos_only(x=2, y="three")

TypeError: pos_only() got some positional-only arguments passed as keyword arguments: 'x, y'

In [27]:
# this works...
pos_only(2, 'three')

"x=2, y='three', z=None"

In [28]:
# this is fine as well...
pos_only(2, "three", 3.14159)

"x=2, y='three', z=3.14159"

## Variable argument lists

- Python allows us to write methods that accept an arbitrary number of positional or keyword arguments without explicitly naming them
    - these arrive as tuple in the function
- we can also accept arbitrary keyword arguments
     - these arrive as dictionary in the function

In [59]:
from urllib.parse import urlparse
from pathlib import Path

def get_pages(*links: str) -> None:
    # arguments passed to links arrive as a tuple
    for link in links:
        url = urlparse(link)
        name = 'index' if url.path in ("", "/") else url.path
        target = Path(url.netloc.replace('.', '_'))
        print(f'Create {target} from {link!r}')
        # etc...

In [60]:
# no parameter
get_pages()

In [61]:
get_pages('https://www.archlinux.org',
          'https://dusty.phillips.codes',
          'https://itmaybeahack.com'
         )

Create www_archlinux_org from 'https://www.archlinux.org'
Create dusty_phillips_codes from 'https://dusty.phillips.codes'
Create itmaybeahack_com from 'https://itmaybeahack.com'


In [71]:
# variable keyword-based arguments
from typing import Any

class Options(dict[str, Any]):
    default_options: dict[str, Any] = {
        "port": 21,
        "host": "localhost",
        "username": None,
        "password": None,
        "debug": False
    }
        
    def __init__(self, **kwargs: Any) -> None:
        super().__init__(self.default_options)
        self.update(kwargs)
        # you can do the following instead of above two statements
        #super().__init__({**self.default_options, **kwargs})
        

In [72]:
options = Options(username="dusty", password="Hunter2", debug=True)

In [73]:
options

{'port': 21,
 'host': 'localhost',
 'username': 'dusty',
 'password': 'Hunter2',
 'debug': True}

## Unpacking arguments

- `*vargs` can be unpacked as tuple or vice-versa
- `**kwargs` can be unpacked as dictionary or vice-versa

In [86]:
t = (1, 2 , 3)
t1 = (3, 4, 5)

In [85]:
print(*t)

1 2 3


In [87]:
# creating t3 tuple by unpacking t and t1
t3 = (*t, *t1)

In [88]:
t3

(1, 2, 3, 3, 4, 5)

In [79]:
x = {'a': 1, 'b': 2}
y = {'b': 11, 'c': 3}

In [80]:
# creating z dictionary by unpacking x and y dicts
# notice key, b
z = {**x, **y}

In [81]:
z

{'a': 1, 'b': 11, 'c': 3}

In [43]:
# try adding * or / before arg3
def show_args(arg1, arg2, arg3="THREE"):
    return f"{arg1=}, {arg2=}, {arg3=}"

In [44]:
some_args = range(3)
show_args(*some_args)

'arg1=0, arg2=1, arg3=2'

In [45]:
more_args = {
    "arg1": 1,
    "arg2": 2
}

In [46]:
show_args(**more_args)

"arg1=1, arg2=2, arg3='THREE'"

## Functions are objects too

- functions may needed to be passed to other functions, objects to perform some tasks
- In Python, functions are already objects!
    - we can set attributes, but not desirable.
- any name defined with `def` keyword is callable objects


In [53]:
def fizz(x: int) -> bool:
    return x%2 == 0

def buzz(x: int) -> bool:
    return x%3 == 0

In [54]:
callable(fizz)

True

In [55]:
callable(buzz)

True

In [56]:
buzz.__name__

'buzz'

In [66]:
from typing import Callable

def name_or_number(number: int, *tests: Callable[[int], bool]) -> str:
    for test in tests:
        if test(number):
            return test.__name__
    return str(number)

In [70]:
name_or_number(1, fizz, buzz)

'1'

In [71]:
name_or_number(3, fizz, buzz)

'buzz'

In [72]:
for i in range(1, 11):
    print(name_or_number(i, fizz, buzz))

1
fizz
buzz
fizz
5
fizz
7
fizz
buzz
fizz


## Function objects and callbacks

- since functions are top-level objects, they can be easily passed around to be exectued in later date or after certain event (event-driven programming)
- callbacks are common as part of building a user interface
    - when the user clicks on something, the framework can call a function so the application code can create a visual response
    - most common in long running tasks, to provide status updates over time; such as downloading/writing/saving large files
- higher order functions that take function as arguments are called callbacks    
- the following examples are adapted from: https://medium.com/analytics-vidhya/understanding-callbacks-a22e8957a73b


In [78]:
def slow_calculation(call_back=None):
    import time
    res = 0
    for i in range(5):
        res += i*i
        time.sleep(1)
        if call_back:
            call_back(i)
    return res

In [79]:
# takes > 5 secs to calculate
slow_calculation()

30

In [76]:
# let's create a call_back function to update the iteration status
def print_with_param(num):
    print(f"Yay!! just completed {num} iteration")

In [77]:
slow_calculation(print_with_param)

Yay!! just completed 0 iteration
Yay!! just completed 1 iteration
Yay!! just completed 2 iteration
Yay!! just completed 3 iteration
Yay!! just completed 4 iteration


30

In [83]:
# using closure
# ability of a function to remember the input to other function
def show_progress_as_function(exclamation):
    def _inner(count):
        print(f'{exclamation} we are in {count} iteration')
    return _inner

In [84]:
f1 = show_progress_as_function('Awesome!!')

In [85]:
f1(1)

Awesome!! we are in 1 iteration


In [86]:
# let's pass f1 as the call back
slow_calculation(f1)

Awesome!! we are in 0 iteration
Awesome!! we are in 1 iteration
Awesome!! we are in 2 iteration
Awesome!! we are in 3 iteration
Awesome!! we are in 4 iteration


30

In [87]:
# since call back only takes 1 argument, we can only pass callback that takes one argument
# chaning two argument function to 1 argument function is common practice in Python

def show_progress(exclamation, iteration):
    print(f'{exclamation} we are in {iteration} iteration')

In [88]:
show_progress('Hooray!!', 1)

Hooray!! we are in 1 iteration


In [89]:
# passing show_progress to slow_calculation will throw an error missing an argument
# work around... using partial
slow_calculation(show_progress)

TypeError: show_progress() missing 1 required positional argument: 'iteration'

## Functools library

- hihger-order functions and operations on callable objects
    - function that act on or return other functions
- https://docs.python.org/3/library/functools.html

In [127]:
# fix
from functools import partial

In [128]:
f1 = partial(show_progress, 'Hooray!!')

In [129]:
f1(2)

Hooray!! we are in 2 iteration


In [131]:
slow_calculation(f1)

Hooray!! we are in 0 iteration
Hooray!! we are in 1 iteration
Hooray!! we are in 2 iteration
Hooray!! we are in 3 iteration
Hooray!! we are in 4 iteration


30

## Using functions to patch a class

- not a usual; but could be useful in special situation
    - **monkey patching** is used in automated testing, e.g.
- can replace/patch objects's methods with your functions

In [90]:
class A:
    def say_something(self):
        print("My class is A")

In [91]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'say_something': <function __main__.A.say_something(self)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [92]:
a = A()
a.say_something()

My class is A


In [93]:
# classes are callable
# we need to call class to create objects
callable(A)

True

In [94]:
# however, objects are not callable!
callable(a)

False

In [95]:
# say_something method is callabe
callable(A.say_something)

True

In [100]:
def patched_say_something():
    print('My class is NOT A')

In [101]:
callable(patched_say_something)

True

In [102]:
a.say_something = patched_say_something

In [103]:
a.say_something()

My class is NOT A


## Callable objects

- just like functions, it is possible to create an object that can be called as though it were a function
- an object can be made callable by giving it a `__call__()` method that accepts required arguments

In [104]:
class Counter:
    def __init__(self):
        self.count = 0
        
    def __call__(self):
        self.count += 1
        

In [105]:
c = Counter()

In [106]:
c.count

0

In [107]:
callable(c)

True

In [108]:
c()

In [109]:
c.count

1

In [110]:
# we can define a callabe object and pass it as a callback to a function
class ProgressBar:
    def __init__(self, exclamation):
        self.exclamation = exclamation
        
    def __call__(self, counter):
        print(f'{self.exclamation} we are in {counter} iteration')

In [113]:
ProgressBar('Hurray')(1)

Hurray we are in 1 iteration


In [114]:
# let's pass ProgressBar object to slow_calculation
slow_calculation(ProgressBar('Yahoo!!'))

Yahoo!! we are in 0 iteration
Yahoo!! we are in 1 iteration
Yahoo!! we are in 2 iteration
Yahoo!! we are in 3 iteration
Yahoo!! we are in 4 iteration


30

## File I/O

- important features of programming languages and software projects
- use `open` built-in function to open file for various modes
- steps:
    1. open a file to read/write/append
    2. do the operation
    3. close the file 
- closing the file is an important step that one may forget

In [157]:
help(open)

Help on function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise OSError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position).
    In

In [165]:
# traditional way to open a file
file = open('filename.txt', 'w')
contents = "Some file contents\n"
file.write(contents)
# what happens if an expcetion is raised in line 4?
file.close()

In [160]:
file = open('filename.txt')
data = file.read()
# what happens if an exception is raised in line 3?
file.close()

In [161]:
print(data)

Some file contents



In [162]:
# reading and writing with context manager
# Note the difference!
with open('filename.txt', 'a') as output:
    output.write('more new contents\n')
    output.write('one more..\n')

In [164]:
with open('filename.txt') as input:
    for line in input:
        print(line.strip())

Some file contents
more new contents
one more..


## Context managers

- Python file objects are context managers
- by using `with` statement, the context management methods ensure that the file is closed even if the exception is raised
- context managers have many applications:
    - when opening files, database or network connections
    - esp. any place where external, operating system-managed resources are involved
- any object can have context manager if needed with appropriate special methods that is used by `with` statements
- let's extend the list cass to create a simple context manager that allows us to construct a sequence of characters and automatically convert it to a string upon exit
    - string is immutable type; so converting it into list of characters is an efficient way to make changes in string
- by defining `__enter__()` and `__exit__()` methods any class can be used as a context manager

- detail explanation and demos can be found here: https://realpython.com/python-with-statement/

In [140]:
# extends list class

class StringJoiner(list):
    def __enter__(self):
        print('context entered...')
        # returned list is bound to the "with" target variable
        return self
    
    # if an exception occurs, inside the  with block,
    # they will be set to values related to the type, value and traceback for the exception
    def __exit__(self, ex_type, ex_value, ex_tb):
        self.result = ''.join(self)
        print('context exiting...')
        return False # this will make sure exception raised is seen
        # change it to True and you'll not see exception in the following example

In [141]:
# sj list is initialized with '[H, e, l, l, o]'
with StringJoiner('Hello') as sj:
    #print(sj) # sj is a list of chars built from Hello
    sj.append(', ')
    sj.append('world')
    sj.append('!')

context entered...
context exiting...


In [142]:
sj

['H', 'e', 'l', 'l', 'o', ', ', 'world', '!']

In [143]:
sj.result

'Hello, world!'

In [144]:
# __exit__ is always executed even there's an exception
with StringJoiner('Partial List') as sj:
    sj.append(" ")
    sj.extend("Results")
    sj.append(str(2/0))
    sj.extend("Even if there's an exception")

context entered...
context exiting...


ZeroDivisionError: division by zero

In [145]:
sj.result

'Partial List Results'

## Using contextmanager decorator

- the above example is stateful context manager
- we can use a function to separate the state-changing object from the context manager that makes the state change
- **@contextmanager** decorator is used to add some features around function to make it work like a context manager class definition 

In [1]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")

In [4]:
with hello_context_manager() as hello:
    # print(hello) if you want to print the yielded string/result
    pass

Entering the context...
Leaving the context...


In [146]:
class StringJoiner2(list[str]):
    def __init__(self, *args: str) -> None:
        super().__init__(*args)
        self.result = "".join(self)

In [153]:
# functional context manager that takes care of steps for entering the context and exiting it

# contextmanager is a also a generator
@contextmanager
def joiner(*args):
    # called on __enter__
    string_list = StringJoiner2(*args)
    try:
        print('yielding object...')
        yield string_list
    finally:
        string_list.result = ''.join(string_list)

In [154]:
with joiner('Hello') as join:
    join.append('Good')
    join.extend('Bye!')

yielding object...


In [149]:
join

['H', 'e', 'l', 'l', 'o', 'Good', 'B', 'y', 'e', '!']

In [150]:
join.result

'HelloGoodBye!'

In [151]:
with joiner('Hello') as join:
    join.append('Good')
    join.append(str(2/0)) #ZeroDivisionError
    join.extend('Bye!')

ZeroDivisionError: division by zero

In [152]:
join.result

'HelloGood'

In [161]:
# providing context to traditional file io
@contextmanager
def writable_file(file_path):
    file = open(file_path, mode="w")
    try:
        yield file
    finally:
        file.close()

In [162]:
with writable_file("context.txt") as file:
    file.write('Hello World from context manager...')

In [164]:
# let's check the contents of the file just created
! cat context.txt

Hello World from context manager...